In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

/home/ubuntu/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
qadata = pd.read_csv("TruthfulQA.csv")

In [5]:
true_sentences = []
false_sentences = []

for row in qadata.iloc:
    corrects = row["Correct Answers"].split("; ")
    incorrects = row["Incorrect Answers"].split("; ")

    for correct in corrects:
        true_sentences.append(f"Q: {row['Question']}\nA: {correct}")

    for incorrect in incorrects:
        false_sentences.append(f"Q: {row['Question']}\nA: {incorrect}")

In [6]:
true_data = model.tokenizer(true_sentences, padding=True, return_tensors="pt").input_ids.to(device)
false_data = model.tokenizer(false_sentences, padding=True, return_tensors="pt").input_ids.to(device)

In [7]:
out, cache = model.run_with_cache(true_data[:16])

In [9]:
torch.argmax(out, shape=-1)

torch.Size([16, 88, 50257])

In [23]:
print(torch.tensor(data["input_ids"]))

tensor([[   48,    25,  1867,  ..., 50256, 50256, 50256],
        [   48,    25,  1867,  ..., 50256, 50256, 50256],
        [   48,    25,  1867,  ..., 50256, 50256, 50256],
        ...,
        [   48,    25,  8920,  ..., 50256, 50256, 50256],
        [   48,    25,  8920,  ..., 50256, 50256, 50256],
        [   48,    25,  8920,  ..., 50256, 50256, 50256]])
